In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,803 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,062 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 4,201 kB in 2s (

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-06-19 04:58:00--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2023-06-19 04:58:01 (10.6 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [4]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [13]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz'
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Outdoors_v1_00.tsv.gz"), inferSchema=True, sep='\t')
df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [14]:
# Get the number of rows in the DataFrame.
row_count = df.count()
print("Number of rows: ", row_count)


Number of rows:  2302401


# Transform the Data

## Create the "review_id_table".

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date

# Create a SparkSession
spark = SparkSession.builder.appName("Create_review_id_df").getOrCreate()

# Select the required columns and transform the data types
review_id_df = df.select(
    df["review_id"],
    df["customer_id"],
    to_date(df["review_date"], "yyyy-MM-dd").alias("review_date"),
    df["product_id"],
    df["star_rating"]
)

# Show the review_id_df DataFrame
review_id_df.show()


+--------------+-----------+-----------+----------+-----------+
|     review_id|customer_id|review_date|product_id|star_rating|
+--------------+-----------+-----------+----------+-----------+
|R35T75OLUGHL5C|   18446823| 2015-08-31|B000NV6H94|          4|
|R2BV735O46BN33|   13724367| 2015-08-31|B000IN0W3Y|          5|
|R2NBEUGPQQGXP1|   51001958| 2015-08-31|B008RBJXFM|          4|
|R17LLAOJ8ITK0S|   32866903| 2015-08-31|B00FK8WUQY|          3|
|R39PEQBT5ISEF4|   30907790| 2015-08-31|B00EZA3VW0|          1|
|R3GNM3SU9VHJFT|   20232229| 2015-08-31|B006JA8WEG|          4|
| R2Y81OP0EK467|   17698862| 2015-08-31|B002PWFSEO|          5|
|R2LFGSI6HAYH5F|   38486114| 2015-08-31|B002DZGKHW|          5|
|R297G6ED1IQO7W|   26319572| 2015-08-31|B00ABA08F6|          5|
| RE27RFC6101N6|   27152337| 2015-08-31|B003Z8WIHC|          5|
|R3BPDME6E94W8Z|   12516845| 2015-08-31|B007CP6UK0|          5|
|R2P08O1RILUOX3|    3225242| 2015-08-31|B003V3U9JK|          3|
|R37CVAB03PTDVI|     961839| 2015-08-31|

## Create the "products" Table

In [16]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("Create products_df").getOrCreate()

# Assuming you have already read the data into a DataFrame called "df"
# Drop duplicates based on "product_id" and "product_title" columns
products_df = df.dropDuplicates(["product_id", "product_title"])

# Show the products_df DataFrame
products_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7501922|R2DQ8GO147LZ8B|0316287229|     253341335|Bike Pedals for K...|        Outdoors|          5|            0|          0|   N|                Y|Works great! i wo...|my son broke his ...| 2015-06-21|
|         US|   40178033|R3DW1PDRX0K6S2|059445039X|     366507607|Blackwatch Plaid ...|        Outdoors|          3|    

## Create the "customers" Table

In [17]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product.
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("Create customers_df").getOrCreate()

# Assuming you have already read the data into a DataFrame called "df"
# Group the data by "customer_id" and count the number of reviews for each customer
customers_df = df.groupBy("customer_id").count()

# Show the customers_df DataFrame
customers_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   43679767|    1|
|   32024654|    1|
|   52913169|    1|
|   24297214|    1|
|   26096454|    1|
|   38247118|    1|
|   44009906|    1|
|    1753876|    1|
|   26195644|    1|
|   23042837|    1|
|   11487525|    2|
|   28258386|    1|
|   42560427|    2|
|   24540309|    1|
|    1967239|    1|
|   14217455|    1|
|   20289484|   26|
|   47103434|    1|
|   28638887|    1|
|     740678|    1|
+-----------+-----+
only showing top 20 rows



## Create the "vine_table".

In [18]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns.
# Create a SparkSession
spark = SparkSession.builder.appName("Create vine_df").getOrCreate()

# Assuming you have already read the data into a DataFrame called "df"
# Select the desired columns to create the vine_df DataFrame
vine_df = df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")

# Show the vine_df DataFrame
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R35T75OLUGHL5C|          4|            0|          0|   N|
|R2BV735O46BN33|          5|            0|          0|   N|
|R2NBEUGPQQGXP1|          4|            0|          0|   N|
|R17LLAOJ8ITK0S|          3|            1|          1|   N|
|R39PEQBT5ISEF4|          1|            0|          0|   N|
|R3GNM3SU9VHJFT|          4|            1|          1|   N|
| R2Y81OP0EK467|          5|            0|          0|   N|
|R2LFGSI6HAYH5F|          5|            1|          1|   N|
|R297G6ED1IQO7W|          5|            1|          1|   N|
| RE27RFC6101N6|          5|            0|          0|   N|
|R3BPDME6E94W8Z|          5|            0|          0|   N|
|R2P08O1RILUOX3|          3|            0|          0|   N|
|R37CVAB03PTDVI|          5|            0|          1|   N|
| RAWNWOGXPCPMD|          5|            

# Load

In [11]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [49]:
# Write the DataFrame to the RDS table
reviews_df.write.jdbc(url=jdbc_url, table=table, mode=mode, properties=config)

Py4JJavaError: ignored

In [33]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='shoes_products_df', mode=mode, properties=config)

Py4JJavaError: ignored

In [34]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_products_df', mode=mode, properties=config)

Py4JJavaError: ignored

In [35]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_products_df', mode=mode, properties=config)

Py4JJavaError: ignored